In [1]:
from __future__ import print_function

import tensorflow as tf
import random

In [2]:
## read text data from file

import csv
import string

labelList = []
tweetList = []
tweetList_test = []

with open('train.csv', encoding='utf-8') as trainfile:
    reader = csv.reader(trainfile)
    for row in reader:
        if row[0] == 'HillaryClinton':
            labelList.append(1)
        elif row[0] == 'realDonaldTrump':
            labelList.append(0)
            
        if row[1] != 'tweet':
            tweetList.append(row[1])
            
with open('test.csv', encoding='utf-8') as testfile:
    reader = csv.reader(testfile)
    for row in reader: 
        if row[1] != 'tweet':
            tweetList_test.append(row[1])

N = len(tweetList)
print('N = %d' % len(tweetList))
print('N = %d' % len(labelList))

N_test = len(tweetList_test)
print('N_test = %d' % len(tweetList_test))

##############################################################

## corpus
import string

def refineWordsList(tweetList):
    ret = []
    for tweet in tweetList:
        step1 = tweet.split('http',1)[0]
        step2 = step1.lower()
        step3 = step2.translate(str.maketrans('.',' '))
        step4 = step3.translate(str.maketrans('','',string.punctuation))
        ret.append(step4)
    return ret

tweetList = refineWordsList(tweetList)
tweetList_test = refineWordsList(tweetList_test)

corpus = set()

for tweet in tweetList:
    words = tweet.split();
    corpus.update(words)

corpus = list(corpus)
M = len(corpus)
print('M = %d' % M)

##############################################################

## Label Encoding

hashEncoder = {}
i = 0
for word in corpus:
    hashEncoder[word] = i
    i += 1

# train
tweetList_encoded = []
for tweet in tweetList:
    tweet_encoded = []
    for word in tweet.split():
        word_encoded = hashEncoder[word]
        tweet_encoded.append(word_encoded)
    tweetList_encoded.append(tweet_encoded)
    
# test
tweetList_test_encoded = []
for tweet in tweetList_test:
    tweet_encoded = []
    for word in tweet.split():
        if hashEncoder.get(word) != None :
            word_encoded = hashEncoder[word]
            tweet_encoded.append(word_encoded)
    tweetList_test_encoded.append(tweet_encoded)

def decodeTweet(tweet_encoded):
    decodeList = []
    for word_encoded in tweetList_encoded[5]:
        decodeList.append(list(hashEncoder.keys())[list(hashEncoder.values()).index(word_encoded)])
    return decodeList
    
print('N = %d' % len(tweetList_encoded))
print('N_test = %d' % len(tweetList_test_encoded))
print()
print(tweetList[5])
print(tweetList_encoded[5])
#print(decodeTweet(tweetList_encoded[5]))
print()
print(tweetList_test[5])
print(tweetList_test_encoded[5])
print()

##############################################################

N_train = (int)(0.8 * N)
N_valid = N - (int)(0.8 * N) - 1

tweetList_encoded_train = [tweetList_encoded[x] for x in range (N_train)]
tweetList_encoded_valid = [tweetList_encoded[x] for x in range (N_train + 1, N)]

labelList_train = [labelList[x] for x in range (N_train)]
labelList_valid = [labelList[x] for x in range (N_train + 1, N)]

print('N_train = %d' % N_train)
print('N_valid = %d' % N_valid)

N = 4743
N = 4743
N_test = 1701
M = 8787
N = 4743
N_test = 1701

when donald trump goes low   register to vote 
[2179, 3482, 5531, 1065, 6065, 7920, 1742, 1085]

its nationalvoterregistrationday  celebrate by registering to vote → 
[7438, 1888, 4778, 6346, 6413, 1742, 1085, 795]

N_train = 3794
N_valid = 948


In [3]:
class SequenceData(object):
    
    def __init__(self, n_samples=1000, max_seq_len = 31, x_list=[], t_list=[]):
        
        self.data = []
        self.labels = []
        self.seqlen = []
        
        for i in range(n_samples):
            len1 = len(x_list[i])
            self.seqlen.append(len1)
            s = [[x_list[i][j]] for j in range(len1)]
            s += [[0] for j in range(max_seq_len - len1 )]
            self.data.append(s)
            self.labels.append([t_list[i], 1 - t_list[i]])
            
        self.batch_id = 0

    def next(self, batch_size):

        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen

In [4]:
# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.01
training_steps = 1000
batch_size = 128
display_step = 100

# Network Parameters
seq_max_len = 31 # Sequence max length
n_hidden = 31 #todo?? hidden layer num of features
n_classes = 2 # linear sequence or notstate_size = 1#20

state_size = 1#20

trainset = SequenceData(n_samples = N_train,
                           max_seq_len = seq_max_len,
                           x_list = tweetList_encoded_train,
                           t_list = labelList_train)

testset = SequenceData(n_samples = N_valid,
                          max_seq_len = seq_max_len,
                          x_list = tweetList_encoded_valid,
                          t_list = labelList_valid)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, state_size])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [5]:
def dynamicRNN(x, seqlen, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

In [6]:
pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

d:\anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))


Step 1, Minibatch Loss= 1.139106, Training Accuracy= 0.67969
Step 100, Minibatch Loss= 0.686262, Training Accuracy= 0.60938
Step 200, Minibatch Loss= 0.629039, Training Accuracy= 0.68750
Step 300, Minibatch Loss= 0.877589, Training Accuracy= 0.00000
Step 400, Minibatch Loss= 0.677515, Training Accuracy= 0.60156
Step 500, Minibatch Loss= 0.626252, Training Accuracy= 0.68750
Step 600, Minibatch Loss= 0.867148, Training Accuracy= 0.00000
Step 700, Minibatch Loss= 0.677670, Training Accuracy= 0.60156
Step 800, Minibatch Loss= 0.625474, Training Accuracy= 0.68750
Step 900, Minibatch Loss= 0.866599, Training Accuracy= 0.00000
Step 1000, Minibatch Loss= 0.677731, Training Accuracy= 0.60156
Optimization Finished!
Testing Accuracy: 0.0021097
